<a href="https://colab.research.google.com/github/ritambara89/wnr/blob/master/Modak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
import matplotlib.pyplot as plt
from IPython.display import Image

Using TensorFlow backend.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [8]:
train = '/content/drive/My Drive/modak/train/'
valid = '/content/drive/My Drive/modak/val/'
batch_size = 32

train_data = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# only rescaling
test_data = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        traindir,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validdir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 2107 images belonging to 2 classes.
Found 232 images belonging to 2 classes.


In [13]:
backbone = VGG16(include_top=False, input_shape=(224, 224, 3))
for layer in backbone.layers:
  layer.trainable = False

model = Sequential()
model.add(backbone)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 21,203,521
Trainable params: 6,488,833
Non-trainable params: 14,714,688
__________________________________

In [0]:

checkpoint = ModelCheckpoint(filepath='modak_weights.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='max',restore_best_weights=True)

In [12]:
epochs=10
history = model.fit_generator(train_generator,
                    epochs=epochs,
                    steps_per_epoch=2107 // batch_size,
                    validation_data=validation_generator,
                    validation_steps=232 // batch_size,
                    callbacks=[checkpoint,early_stopping],
                    verbose=1
                   )

Epoch 1/10
 2/65 [..............................] - ETA: 1:08 - loss: 0.0454 - acc: 0.9844

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


33/65 [==============>...............] - ETA: 24s - loss: 0.0981 - acc: 0.9650

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


65/65 [==============================] - 46s 712ms/step - loss: 0.1072 - acc: 0.9599 - val_loss: 0.0909 - val_acc: 0.9688

Epoch 00001: val_acc did not improve from 0.97000
Epoch 2/10
65/65 [==============================] - 46s 712ms/step - loss: 0.0934 - acc: 0.9625 - val_loss: 0.0561 - val_acc: 0.9700

Epoch 00002: val_acc did not improve from 0.97000
Epoch 3/10
65/65 [==============================] - 46s 714ms/step - loss: 0.0936 - acc: 0.9625 - val_loss: 0.0376 - val_acc: 0.9850

Epoch 00003: val_acc improved from 0.97000 to 0.98500, saving model to modak_best_weights.h5
Epoch 4/10
65/65 [==============================] - 46s 711ms/step - loss: 0.1159 - acc: 0.9546 - val_loss: 0.0879 - val_acc: 0.9650

Epoch 00004: val_acc did not improve from 0.98500
Epoch 5/10
65/65 [==============================] - 45s 691ms/step - loss: 0.0439 - acc: 0.9884 - val_loss: 0.0758 - val_acc: 0.9700

Epoch 00005: val_acc did not improve from 0.98500
Epoch 6/10
65/65 [==============================